In [1]:
import pandas as pd
import numpy as np

In [2]:
import re

In [3]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('fivethirtyeight')

In [4]:
import networkx as nx

In [5]:
import sys

sys.path.append('C:\\Users\\pradhyum\\github_repos\\TwitterRank_new')
import TwitterRank


In [ ]:
tweets_df = pd.read_csv('AI Twitter.csv',delimiter=',',skiprows=6,error_bad_lines=False)

In [ ]:
tweets_df.head()

In [ ]:
tweets_df.info()

In [ ]:
tweets_df.columns

In [ ]:
def remove_links(tweet):
    tweet = re.sub('http\S+','',tweet)
    tweet = re.sub('(?!http://)bit.ly/\S+','',tweet)
    tweet = re.sub('[A-Za-z0-9.@]+\.[A-Za-z0-9]+/[A-Za-z0-9.@]+','',tweet)
    tweet = tweet.strip('[link]')
    return tweet
def extract_links(tweet):
    link1 = re.findall('http\S+',tweet)
    return list(set(link1.extend(re.findall('(?!http://)bit.ly\S+',tweet))))
def remove_users(tweet):
    tweet = re.sub('@[A-Za-z]+[A-Za-z0-9-_]','',tweet)
    return tweet
def remove_hashtags(tweet):
    tweet = re.sub('(#[A-Za-z]+[A-Za-z0-9-_]+)','', tweet)
    return tweet

In [ ]:
import nltk 
import spacy
nlp = spacy.load('en_core_web_sm')
nlp.Defaults.stop_words.add('rt')
nlp.vocab['rt'].is_stop = True
my_punctuation = '!"$%&\'()*+,-./:;<=>?[\\]^_`{|}~•@'

# cleaning master function
def clean_tweet(tweet, bigrams=False):
    tweet = remove_users(tweet)
    tweet = remove_links(tweet)
    tweet = remove_hashtags(tweet)
    tweet = tweet.lower() # lower case
    tweet = re.sub('['+my_punctuation + ']+', ' ', tweet) # strip punctuation
    tweet = re.sub('\s+', ' ', tweet) #remove double spacing
    tweet = re.sub('([0-9]+)', '', tweet) # remove numbers
    tweet_token_list = [word for word in nlp(tweet)
                            if not nlp.vocab[word.text].is_stop] # remove stopwords

    tweet_token_list = [word.lemma_ if '#' not in word.text else word.text
                        for word in tweet_token_list] # apply lemmatization
    tweet = ' '.join(tweet_token_list)
    return tweet

In [ ]:
tweets_df['Clean Tweet'] = tweets_df['Full Text'].apply(clean_tweet)

In [ ]:
def model_topics(algorithm, vocab, model_in=None):
    if model_in is None:
        if algorithm == 'LDA':
            model = LatentDirichletAllocation(n_components=10)
        elif algorithm == 'NMF':
            model = NMF(n_components=10)
        model.fit(tf)
        model.get_params()
    else: model = model_in
    topic_word_df = pd.DataFrame(model.components_,columns=vocab)
    sorted_topic_words = pd.DataFrame()
    for index, row in topic_word_df.iterrows():
        row_df = pd.DataFrame({'topic_'+str(index): row.sort_values(ascending=False).index[:5].values})
        sorted_topic_words = pd.concat([sorted_topic_words,row_df],axis=1)
    return topic_word_df, sorted_topic_words

In [ ]:
def find_retweet_of(tweet):
    tweet = re.findall('(?<=http://twitter.com/)\w+',tweet)[0]
    return tweet
tweets_df['Retweet of'] = tweets_df['Twitter Retweet of'].apply(lambda x:find_retweet_of(x) if not pd.isna(x) else x)

In [ ]:
tweets_df.to_csv('AI_twitter_cleaned.csv',index=False)

In [6]:
tweets_df = pd.read_csv('AI_twitter_cleaned.csv')

In [ ]:
#tweets_df['Retweet of']

In [ ]:
#G = nx.from_pandas_edgelist(tweets_df[tweets_df['Retweet of'].notna()],'Retweet of','Author')

In [ ]:
#G.remove_nodes_from([i for i in list(G.nodes()) if i not in tweets_df['Author'].values])

In [ ]:
#from scipy.sparse import csr_matrix, lil_matrix
#adj = nx.to_scipy_sparse_matrix(G)
#adj.get_shape()

In [ ]:
#tweets_df.groupby('Author').get_group('Kozinets')['Clean Tweet'].count()

In [ ]:
#tweets_df.groupby('Author').get_group('Kozinets')['Clean Tweet'].count()
import time, sys
from IPython.display import clear_output

def update_progress(progress):
    bar_length = 80
    if isinstance(progress, int):
        progress = float(progress)
    if not isinstance(progress, float):
        progress = 0
    if progress < 0:
        progress = 0
    if progress >= 1:
        progress = 1

    block = int(round(bar_length * progress))

    clear_output(wait = True)
    text = "Progress: [{0}] {1:.1f}%".format( "#" * block + "-" * (bar_length - block), progress * 100)
    print(text)

In [ ]:
#nx_graph = TwitterRank.get_graph_object(tweets_df,filter_column='Retweet of')

In [ ]:
#new_df = pd.DataFrame(tweets_df[tweets_df['Retweet of'].isin(tweets_df['Author'].value_counts().index)].values, columns=tweets_df.columns)

In [ ]:
#len(nx_graph)

In [ ]:
#nodes_to_remove = tweets_df['Retweet of'][~tweets_df['Retweet of'].isin(tweets_df['Author'].value_counts().index)].unique()

In [ ]:
#nx_graph.remove_nodes_from(list(nodes_to_remove))

In [ ]:
#ser = pd.Series(tweets_df['Author'].unique()).isin(pd.Series(nx_graph.nodes()))
#ser[ser==False].index

In [ ]:
#tweets_df.drop([53440],axis=0,inplace=True)

In [ ]:
#tweets_list = TwitterRank.get_num_tweets_list(nx_graph,tweets_df)

In [ ]:
#relationship = TwitterRank.get_relationship(nx_graph)

In [ ]:
#friend_tweet_list = TwitterRank.get_friends_tweets_list(relationship, tweets_list)

In [ ]:
#model, vocab_list, term_frequency = TwitterRank.get_lda_model(5, 30, tweets_df)
#dt = np.mat(model._unnormalized_transform(term_frequency))
#row_normalized_dt = dt/np.sum(dt,axis=0)

In [ ]:
#pt = TwitterRank.get_Pt(1,tweets_list,friend_tweet_list,row_normalized_dt,relationship)

In [ ]:
#Et = dt/np.sum(dt,axis=1)

In [ ]:
#Et[:,0].shape

In [ ]:
#pd.DataFrame(TRt,index=nx_graph.nodes(),columns=['topic1 influence score']).sort_values(by='topic1 influence score',ascending=False).head(10)

In [ ]:
#TwitterRank.get_TR_using_DT(dt, tweets_df, num_topics=5, gamma=0.2, tolerance=1e-16)

In [ ]:
#col_normalized_dt = dt/np.sum(dt,axis=1)

In [7]:
tweets_df['Clean Tweet'] = tweets_df['Clean Tweet'].apply(lambda x:str(x))

In [10]:
model_, vocab_, tf_ = TwitterRank.get_lda_model(5,10,tweets_df)
dt = model_._unnormalized_transform(tf_)

Creating term-author matrix
Fitting LDA model to discover topics


In [11]:
TR, graph = TwitterRank.get_TR_using_DT(dt, tweets_df, 5, gamma=0.2, tolerance=1e-16)

Creating graph object
Removing nodes where author doesn't exist in raw df
Creating relationship matrix
Gathering tweet count for all users
Calculating influence scores for users under topic 0
topic number 1
Creating transition probability for topic 1
Calculating influence scores for users under topic 1
topic number 2
Creating transition probability for topic 2
Calculating influence scores for users under topic 2
topic number 3
Creating transition probability for topic 3
Calculating influence scores for users under topic 3
topic number 4
Creating transition probability for topic 4
Calculating influence scores for users under topic 4


In [12]:
#TR, graph, lda_model, lda_vocab = TwitterRank.twitter_rank(tweets_df)

In [13]:
final_ranks = TwitterRank.get_top_topic_influencers(TR,graph)
final_ranks

,Topic0Influncers,Topic1Influncers,Topic2Influncers,Topic3Influncers,Topic4Influncers
0,i_UCSY,FireTap_Digital,WitchyTheThird,onmedicglobal,matthewherper
1,surajpramanik5,abhay16dec,Ohrkid,kbeguir,TeamATSG
2,mapanauta,Camtheagent,AyyoubAkbari,storytellr2002,Shwento
3,TechWorldOracle,BrainCrumbz,arnaldoarnalm,cardiogalenico,abhay16dec
4,AaceQatar,matthewherper,sloaneguy,abhay16dec,TrapObi
5,azizakhan8,KayandNnn,stephenjaoates,_HeatherE,jmannteufel
6,hikarenchan,hellojabez,SPCOsandiego,ikoha254,KayandNnn
7,matthewherper,TK_CH291,WaterSolarWind,RajenderSanti,AyyoubAkbari
8,OneGameDad,ZebraCakes,otonielgonca,ZebraCakes,Abbyroad713
9,milancermak,mwesterl,DeannaBurgart,TK_CH291,JamesHubbard97


In [16]:
topics = TwitterRank.print_topics_as_df(model_,vocab_)
topics

,topic_0,topic_1,topic_2,topic_3,topic_4
0,face,machine,vote,ai,ai
1,image,learn,recognition,google,ibm
2,network,ai,intel,machine,google
3,medium,google,want,microsoft,intelligence
4,nvidia,learning,facial,intelligence,artificial


In [ ]:
 from matplotlib import pylab
 import networkx as nx

 def save_graph(graph,file_name):
    #initialze Figure
    plt.figure(num=None, figsize=(20, 20), dpi=80)
    plt.axis('off')
    fig = plt.figure(1)
    pos = nx.spring_layout(graph)
    nx.draw_networkx_nodes(graph,pos)
    nx.draw_networkx_edges(graph,pos)
    nx.draw_networkx_labels(graph,pos)

    cut = 1.00
    xmax = cut * max(xx for xx, yy in pos.values())
    ymax = cut * max(yy for xx, yy in pos.values())
    plt.xlim(0, xmax)
    plt.ylim(0, ymax)

    plt.savefig(file_name,bbox_inches="tight")
    pylab.close()
    del fig

#Assuming that the graph g has nodes and edges entered
save_graph(graph,"my_graph.png")

#it can also be saved in .svg, .png. or .ps formats

In [ ]:
#plt.subplots(figsize=(16,16))
#nx.draw(graph)